## A Pythonic object


### str() vs repr()

- repr: Return a string representing the object as the developer wants to see it.
- str: Return a string representing the object as the user wants to see it.



In [37]:
from array import array
import math

class Vector2d:
    
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt=''):
        print(self, type(self))
        components = (format(c, fmt) for c in self)
        return '({}, {})'.format(*components)
    
    

In [2]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [4]:
x,y = v1
x,y

(3.0, 4.0)

In [5]:
v1

Vector2d(3.0, 4.0)

In [6]:
str(v1)

'(3.0, 4.0)'

In [7]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [8]:
ord('d')

100

In [10]:
bytes(100)

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [1]:
# __iter__

class Next:
    
    def __init__(self, seq):
        self.seq = list(seq)
        
    def __iter__(self):
        
        return self
    
    def __next__(self):
        if not self.seq:
            raise StopIteration
        else:
            return self.seq.pop(0)
        

In [2]:
n = Next(range(5))
next(n)

0

In [4]:
n.__iter__()

In [5]:
next(n)

1

In [7]:
for i in n:
    print(i)

2
3
4


### classmethod versus staticmethod


In [12]:
class Demo:
    
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def staticmeth(*args):
        return args
    
    

In [10]:
Demo.klassmeth('zx')

(__main__.Demo, 'zx')

In [13]:
Demo.staticmeth('zx')

('zx',)

- 使用了 classmethod 会把类本身作为一个参数
- staticmethod 不会

### Formatted displays


In [14]:
brl = 1/2.43
brl

0.4115226337448559

In [17]:
format(brl, '0.4f')

'0.4115'

In [19]:
'{:.2f}'.format(brl)

'0.41'

In [20]:
# 进制转换
format(10, 'b')

'1010'

In [21]:
format(10, 'x')

'a'

In [22]:
format(2/3, '.1f')

'0.7'

In [23]:
# 格式化时间

from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'12:31:40'

In [24]:
'it is now : {: %I:%M %p}'.format(now)

'it is now :  12:31 PM'

In [38]:
# 重写了 Vector2d 的 format 方法

v2 = Vector2d(3, 4)
format(v2)

(3.0, 4.0) <class '__main__.Vector2d'>


'(3.0, 4.0)'

In [31]:
format(v2, '.3f')

'(3.000, 4.000)'

In [35]:
print(format(v2, '.3f'))

(3.000, 4.000)


### A hashable Vector2d


In [39]:
# 目前 Vector2d 是无法被 hash 的

v4 = Vector2d(5, 6)
hash(v4)

TypeError: unhashable type: 'Vector2d'

In [41]:
set([v4])

TypeError: unhashable type: 'Vector2d'

In [43]:
# 原因
v4.x = 6
v4

Vector2d(6, 6.0)

In [48]:
# 使 ins.x 不能被改变

class SubVector2d():
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        
        return self.__x
    
    @property
    def y(self):
        return self.__y
    

In [49]:
v5 = SubVector2d(3,4)
hash(v5)

-9223363295199524366

- 测试发现 SubVecter2d 继承 Vecter2d 也是不能被 hash 的
- property 装饰器是将方法转变为属性

### Saving space with the __slots__ class attribute


In [53]:
# 适用场景
# 大量的实例, 成百上千
#　固定的属性

class V:
    
    __slots__ = ('x', 'y')
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
class nomalV:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y


In [51]:
v = V(1,2)
v.x, v.y

(1, 2)

In [56]:
# 使用 slots 后，新增未注册的属性会报错
v.z = 3

AttributeError: 'V' object has no attribute 'z'

In [54]:
nv = nomalV(3,4)
nv.x,nv.y

(3, 4)

In [57]:
# 不会报错 
nv.z = 1
nv.z

1